In [1]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import SE3, SO3
from scipy.interpolate import CubicSpline
from roboticstoolbox.backends.swift import Swift
import matplotlib.pyplot as plt
import time
from spatialgeometry import Cuboid

Robot in the envirnoment

In [2]:
# Initialize Swift 3D visualization environment
env = Swift()

# Launch browser window at http://localhost:9000
# env.launch(realtime=True,comms="rtc",browser="notebook")
env.launch()

# Create Panda robot model
panda = rtb.models.Panda()

# Define home configuration (ready pose, 7 joint angles in radians)
q_home = np.array([0, -np.pi/4, 0, -3*np.pi/4, 0, np.pi/2, np.pi/4])

# Add robot to Swift environment for 3D visualization
env.add(panda)

0

Objects in the environment

In [3]:
# Dimensioni cubo ridotte
cube_size = 0.06  # 6cm invece di 10cm
cube_height = cube_size
cube_center_z = cube_size / 2  # centro a 3cm da terra

# Spacing ridotto proporzionalmente
spacing = 0.12  # 12cm tra centri (era 15cm)

# Altezza per il pick: cima del cubo
pick_z = cube_center_z + cube_height/2  # = 0.03 + 0.03 = 0.06

objects = {
    'red':    (np.array([0.5, 0.25, pick_z]), np.array([0.2, -0.30, pick_z])),
    'blue':   (np.array([0.5, 0.13, pick_z]), np.array([0.2, -0.42, pick_z])),
    'green':  (np.array([0.5, 0.01, pick_z]), np.array([0.2, -0.54, pick_z])),
    'yellow': (np.array([0.5, -0.11, pick_z]), np.array([0.2, -0.54, pick_z]))  # ad es. non -0.66
}

# Print info
print(f"Cube size: {cube_size}m")
print(f"Pick/Place height: z={pick_z:.3f}m")
print(f"Objects to sort: {len(objects)}")

for name, (pick, place) in objects.items():
    print(f"  {name}: pick={pick}, place={place}")

# Create visible cube objects (dimensioni ridotte)
bin_red = Cuboid([cube_size, cube_size, cube_size], 
                 pose=SE3(0.5, 0.25, cube_center_z), color=[1, 0, 0, 0.8])      
bin_blue = Cuboid([cube_size, cube_size, cube_size], 
                  pose=SE3(0.5, 0.13, cube_center_z), color=[0, 0, 1, 0.8])     
bin_green = Cuboid([cube_size, cube_size, cube_size], 
                   pose=SE3(0.5, 0.01, cube_center_z), color=[0, 1, 0, 0.8])    
bin_yellow = Cuboid([cube_size, cube_size, cube_size], 
                    pose=SE3(0.5, -0.11, cube_center_z), color=[1, 1, 0, 0.8])  

# Add to environment
env.add(bin_red)
env.add(bin_blue)
env.add(bin_green)
env.add(bin_yellow)

print("[OK] Objects added to Swift")


Cube size: 0.06m
Pick/Place height: z=0.060m
Objects to sort: 4
  red: pick=[0.5  0.25 0.06], place=[ 0.2  -0.3   0.06]
  blue: pick=[0.5  0.13 0.06], place=[ 0.2  -0.42  0.06]
  green: pick=[0.5  0.01 0.06], place=[ 0.2  -0.54  0.06]
  yellow: pick=[ 0.5  -0.11  0.06], place=[ 0.2  -0.54  0.06]
[OK] Objects added to Swift


# Compute the trajectory that the robot need to do for picking 

## Functions

non molto bene

In [4]:
def compute_ik(panda, position, z_offset=0.0):  # ← metti 0.0 di default
    """Compute IK for a position slightly above the object"""
    target = np.array(position).copy()
    target[2] += z_offset      
    
    pose = SE3(target[0], target[1], target[2]) * SE3.Rx(np.pi)
    
    result = panda.ikine_LM(pose, q0=q_home, joint_limits=True)
    return result.q, result.success


def make_traj(q_start, q_goal, duration=2.0, n_points=100):
    """Generate a smooth joint-space trajectory from q_start to q_goal."""
    t = np.linspace(0, duration, n_points)          # time vector
    traj = np.zeros((n_points, 7))                  # 7 joints
    
    for i in range(7):
        # cubic spline between start and goal for joint i
        cs = CubicSpline([0, duration], [q_start[i], q_goal[i]])
        traj[:, i] = cs(t)
    
    return traj


## Dictionary with trajectory

In [5]:
approach_height = 0.12  # 12 cm sopra la cima del cubo

trajectories = {}

for name, (pick_pos, place_pos) in objects.items():
    # 1) IK for approach and contact poses
    
    # Above pick and at pick
    q_pick_above, ok1 = compute_ik(panda, pick_pos, z_offset=approach_height)
    q_pick,       ok2 = compute_ik(panda, pick_pos, z_offset=0.0)
    
    # Above place and at place
    q_place_above, ok3 = compute_ik(panda, place_pos, z_offset=approach_height)
    q_place,       ok4 = compute_ik(panda, place_pos, z_offset=0.0)
    
    if not (ok1 and ok2 and ok3 and ok4):
        print(f"[ERROR] IK failed for {name}")
        continue

    # 2) Joint-space trajectories for each phase
    traj_home_to_pick_above      = make_traj(q_home,        q_pick_above)
    traj_pick_above_to_pick      = make_traj(q_pick_above,  q_pick)
    traj_pick_to_pick_above      = make_traj(q_pick,        q_pick_above)
    traj_pick_above_to_place_abv = make_traj(q_pick_above,  q_place_above)
    traj_place_abv_to_place      = make_traj(q_place_above, q_place)
    traj_place_to_place_abv      = make_traj(q_place,       q_place_above)
    traj_place_abv_to_home       = make_traj(q_place_above, q_home)

    # 3) Save everything in the dictionary
    trajectories[name] = {
        "home_to_pick_above":       traj_home_to_pick_above,
        "pick_above_to_pick":       traj_pick_above_to_pick,
        "pick_to_pick_above":       traj_pick_to_pick_above,
        "pick_above_to_place_abv":  traj_pick_above_to_place_abv,
        "place_abv_to_place":       traj_place_abv_to_place,
        "place_to_place_abv":       traj_place_to_place_abv,
        "place_abv_to_home":        traj_place_abv_to_home,
    }

## Do the movement

In [6]:
# Map each logical object name to its visual Cuboid in Swift
cube_objs = {
    'red': bin_red,
    'blue': bin_blue,
    'green': bin_green,
    'yellow': bin_yellow
}

In [7]:
def execute_pick_and_place(name, sleep_dt=0.02):
    """
    Execute the joint-space trajectories for one object and
    visually attach/detach the cube to the robot gripper.
    """
    cube = cube_objs[name]      # visual cube object (bin_red, bin_blue, ...)
    trajs = trajectories[name]  # all phases for this object

    # --- 1) Home -> pick_above (robot si avvicina, cubo fermo) ---
    for q in trajs["home_to_pick_above"]:
        panda.q = q
        env.step(0.01)
        time.sleep(sleep_dt)

    # --- 2) pick_above -> pick (discesa, ancora cubo fermo) ---
    #     alla fine calcoliamo la trasformazione relativa mano->cubo
    last_q_contact = trajs["pick_above_to_pick"][-1]
    for q in trajs["pick_above_to_pick"]:
        panda.q = q
        env.step(0.01)
        time.sleep(sleep_dt)

    # Pose della mano e del cubo al momento del "contatto"
    T_ee_contact   = panda.fkine(last_q_contact)  # end-effector pose at contact
    T_cube_contact = cube.T                       # cube pose on the table
    T_rel = T_ee_contact.inv() * T_cube_contact  # rigid transform hand -> cube

    # --- 3) pick -> pick_above (cubo ATTACCATO al gripper) ---
    for q in trajs["pick_to_pick_above"]:
        panda.q = q
        T_ee = panda.fkine(q)
        cube.T = T_ee * T_rel                    # cube follows the hand rigidly
        env.step(0.01)
        time.sleep(sleep_dt)

    # --- 4) pick_above -> place_above (trasporto in aria con cubo attaccato) ---
    for q in trajs["pick_above_to_place_abv"]:
        panda.q = q
        T_ee = panda.fkine(q)
        cube.T = T_ee * T_rel
        env.step(0.01)
        time.sleep(sleep_dt)

    # --- 5) place_above -> place (discesa per appoggiare il cubo) ---
    last_q_place = trajs["place_abv_to_place"][-1]
    for q in trajs["place_abv_to_place"]:
        panda.q = q
        T_ee = panda.fkine(q)
        cube.T = T_ee * T_rel
        env.step(0.01)
        time.sleep(sleep_dt)

    # Forza il cubo esattamente nella posizione di place,
    # ma con z = cube_center_z così NON resta sospeso in aria.
    place_pos = objects[name][1]          # (x, y, z_pick) dal dizionario
    cube.T = SE3(place_pos[0], place_pos[1], cube_center_z)

    # --- 6) place -> place_above (gripper si alza, cubo RESTA sul tavolo) ---
    for q in trajs["place_to_place_abv"]:
        panda.q = q
        # non aggiorniamo più cube.T: il cubo resta dove lo abbiamo lasciato
        env.step(0.01)
        time.sleep(sleep_dt)

    # --- 7) place_above -> home (robot torna a home) ---
    for q in trajs["place_abv_to_home"]:
        panda.q = q
        env.step(0.01)
        time.sleep(sleep_dt)


In [8]:
for name in trajectories.keys():
    print(f"[INFO] Executing pick-and-place for {name}")
    execute_pick_and_place(name, sleep_dt=0.02)

[INFO] Executing pick-and-place for red
[INFO] Executing pick-and-place for blue
[INFO] Executing pick-and-place for green
[INFO] Executing pick-and-place for yellow
